1. Implement two classification algorithms for detecting faces and classifying digits:
(a) Perceptron
(b) Two-layer Neural Network (input layer, one hidden layer, output)
2. Design the features for each of the two problems, and write a program for extracting the features from each image.
You can also use the raw pixels directly as features (and avoid the process of designing features altogether) if that
results in a better classification accuracy.
3. Train the two algorithms on the part of the data set that is reserved for training. First, use only 10% of the data points
that are reserved for training, then 20%, 30%, 40%, 50%, 60%, 70%, 80%, 90%, and finally 100%. All the results
should a function of the number of data points used for training.
4. Compare the performances of the two algorithms using the part of the data set that is reserved for testing, and report:
• The time needed for training as a function of the number of data points used for training.
• The prediction error (and standard deviation) as a function of the number of data points used for training.
5. Write a report describing the implemented algorithms and discussing the results and the learned lessons.

In [10]:
# if f(x) >= 0 then y = face
# else y = not face

#f(x) = w1*phi(x) + w2*phi2(x) + ..
# phi(x) = number of black pixels (example)

# intialize weights randomonly at first

# training condition is while less than 5 minutes or
# the weights aren't changing from image to image

#  update weights in training
 # if f(x) >=0 and y =face
 # or  f(x) <= 0 and y = not face
 # then do nothing

 # else if f(x) >= 0 and y = not face
 # then w1 = w1 -phi1 (x) 
 #      w2 = w2 - phi2(x)

# else if f(x) < 0 and y = face
 # then w1 = w1 + phi1 (x) 
 #      w2 = w2 + phi2(x)


#f = w1 * phi1(x) + w2 * phi2(x)



In [11]:
# Determine features - will need to obtain from image
# First step is get data from facedatatrain to use in this notebook
# Each face image is 70x60 (row x column) and each face begins and ends with an empty line (not including the 70x60).
# Then make a feature matrix of size 70*60, where each pixel is a feature

In [12]:
from Berkeley_Example import samples

In [13]:
RESOLUTION = 4200
face_images = samples.loadDataFile('data/facedata/facedatatrain', 100, 60, 70)
for img in face_images:
  pass
  #print(img.getPixel(0,0))
  

['                                                            ', '                                                            ', '                             ###                            ', '                          ###   ###                         ', '                        ##         ####                     ', '                      ##               ###                  ', '                     #                    ###               ', '                    #                        #              ', '                    #                         #     ###     ', '                #   #                          #   #   #### ', '               #   #                            ###         ', '              #    #          ##                 #          ', '              #    #         #  #                 #         ', '             #     #    #    #   ###               #        ', '             #     #   #     #  #   #              #        ', '            #      # ##      #  #   # 

In [14]:
import numpy as np
weights = np.random.rand(1, 4200)
phitures = 0